In [3]:
import win32com.client
import time
import shutil
import os
import codecs
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from pywinauto import application
from pywinauto import timings

In [4]:
# 0: 날짜(ulong)
# 1:시간(long) - hhmm
# 2:시가(long or float)
# 3:고가(long or float)
# 4:저가(long or float)
# 5:종가(long or float)
# 6:전일대비(long or float) - 주) 대비부호(37)과 반드시 같이 요청해야 함
# 8:거래량(ulong or ulonglong) 주) 정밀도 만원 단위
# 9:거래대금(ulonglong)
# 10:누적체결매도수량(ulong or ulonglong) - 호가비교방식 누적체결매도수량
# 11:누적체결매수수량(ulong or ulonglong) - 호가비교방식 누적체결매수수량
#  (주) 10, 11 필드는 분,틱 요청일 때만 제공
# 12:상장주식수(ulonglong)
# 13:시가총액(ulonglong)
# 14:외국인주문한도수량(ulong)
# 15:외국인주문가능수량(ulong)
# 16:외국인현보유수량(ulong)
# 17:외국인현보유비율(float)
# 18:수정주가일자(ulong) - YYYYMMDD
# 19:수정주가비율(float)
# 20:기관순매수(long)
# 21:기관누적순매수(long)
# 22:등락주선(long)
# 23:등락비율(float)
# 24:예탁금(ulonglong)
# 25:주식회전율(float)
# 26:거래성립률(float)
# 37:대비부호(char) - 수신값은 GetHeaderValue 8 대비부호와 동일

In [5]:
# In[ ]:


class CREON(object):
    """대신증권 크레온 API"""
    
    def __init__(self):
        # 연결 여부 체크
        self.objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = self.objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            try:
                os.system("/Users/juhy9/Documents/GitHub/System/Quant/src/sudo/sudo_creon_plus_login.bat")
                time.sleep(100)
            except:
                exit()
     
    def setMethod(self, code, char, from_yyyymmdd=None, to_yyyymmdd=None, count=None):
        """
        count는 보통 상식의 데이터 개수가 아니다.
        여기서는 한번 요청 시 가져와지는 데이터의 개수이다.
        한번 요청 시 최대 2856개 가능하다.
        
        원하는 데이터 개수가 있으면 to_yyyymmdd 로 가져온 다음에 잘라서 사용한다.
        하루에 분단위 데이터가 381개이다. (* 마지막 10분은 동시호가)
        
        """
        # object 구하기
        self.objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
        self.objStockChart.SetInputValue(0, code)  # 종목코드
        
        if to_yyyymmdd:
            self.objStockChart.SetInputValue(1, ord('1'))  # 요청 구분 '1': 기간, '2': 개수
            self.objStockChart.SetInputValue(2, from_yyyymmdd)  # To 날짜
            self.objStockChart.SetInputValue(3, to_yyyymmdd)  # From 날짜
        elif count:
            self.objStockChart.SetInputValue(1, ord('2'))  # 개수로 받기
            self.objStockChart.SetInputValue(4, count)  # 조회 개수
        else: raise print("기간을 입력해주세요.")
        
        if char == "m":
            # 날짜, 시간,시가,고가,저가,종가,거래량
            self.colnames = "날짜, 시간, 시가, 고가, 저가, 종가, 거래량".split(", ")
            self.objStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8])
        else:
            # 날짜,시가,고가,저가,종가,거래량, 거래대금, 상장주식수, 시가총액, 외국인현보유수량, 기관순매수
            self.colnames = "날짜, 시가, 고가, 저가, 종가, 거래량, 거래대금, 상장주식수, 시가총액, 외국인현보유수량, 기관순매수".split(", ")
            self.objStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8, 9, 12, 13, 16, 20])
            
        self.objStockChart.SetInputValue(6, ord(char))  # '차트 주기 - 분/틱
        self.objStockChart.SetInputValue(7, 1)  # 분틱차트 주기
        
        self.objStockChart.SetInputValue(9, ord('1'))  # 수정주가 사용
        
        
        
        self.data = {i: [] for i in self.colnames}
        
    def checkRequest(self):
        
        self.objStockChart.BlockRequest()
        
        rqStatus = self.objStockChart.GetDibStatus()
        
        if rqStatus != 0: 
            
            return False
        
#         else:
#             print("통신상태 양호, 누적 개수 {}".format(len(self.data["date"])))
        
        self.count = self.objStockChart.GetHeaderValue(3)
        
        if self.count <= 1: 
            
            return False
        
        return int(self.count)
    
    def checkRemainTime(self):
        
        # 연속 요청 가능 여부 체크
        remainTime = self.objCpCybos.LimitRequestRemainTime / 1000.
        remainCount = self.objCpCybos.GetLimitRemainCount(1)  # 시세 제한
        
        if remainCount <= 0:
            print("15초당 60건으로 제한합니다.")
            time.sleep(remainTime)
            
    
    def getStockPriceMin(self):
        
        while 1:
        
            self.checkRemainTime()
            rows = self.checkRequest()

            if rows:

                for i in range(rows):
                    
                    for idx, col in enumerate(self.colnames):
                    
                        self.data[col].append(self.objStockChart.GetDataValue(idx, i))
            else:

                break
                
    
        return self.data

In [6]:
KoreaStock = pd.read_csv("/Users/juhy9/Documents/GitHub/System/Quant/src/db/stock_info_20200703.csv")
KoreaStock

,종목코드,상장주식수20억이상,증권전산업종코드,소속부,감리구분,주식상태,자본금규모구분,KOSPI200종목여부,부구분코드,종목명,...,분기경상이익,분기당기순이익,분개매출액영업이익률,분기매출액경상이익률,분기ROE,분기이자보상비율,분기유보율,분기부채비율,시가총액,PBR
0,U001,1,0,구분없음,정상,정상,제외,미채용,구분없음,KOSPI지수,...,0,0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
1,A000020,1,9,거래소,정상,정상,소,미채용,주권,동화약품,...,3268000000,2258000000,4.04,4.88,3.01,45.790001,981.250000,25.610001,4.203686e+11,1.403656
2,A000040,1,15,거래소,정상,정상,소,미채용,주권,KR모터스,...,-850000000,-850000000,1.35,-2.98,-17.00,0.430000,0.000000,341.079987,6.278780e+10,2.008798
3,A000050,1,16,거래소,정상,정상,소,미채용,주권,경방,...,-2336000000,-3167000000,4.29,-3.16,-1.72,1.660000,5348.740234,91.870003,3.207587e+11,0.437629
4,A000060,1,25,거래소,정상,정상,중,미채용,주권,메리츠화재,...,146244000000,103620000000,5.08,5.08,16.82,0.000000,4296.419922,880.669983,1.472156e+12,0.600900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,Q590013,1,0,거래소,정상,정상,제외,미채용,None,미래에셋 미국 리,...,0,0,0.00,0.00,0.00,0.000000,0.000000,0.000000,1.883000e+10,0.000000
1547,Q590016,1,0,거래소,정상,정상,제외,미채용,None,미래에셋 미국 헬,...,0,0,0.00,0.00,0.00,0.000000,0.000000,0.000000,2.818000e+10,0.000000
1548,Q590017,1,0,거래소,정상,정상,제외,미채용,None,미래에셋 미국 고,...,0,0,0.00,0.00,0.00,0.000000,0.000000,0.000000,2.567000e+10,0.000000
1549,Q590018,1,0,거래소,정상,정상,제외,미채용,None,미래에셋 중국 심,...,0,0,0.00,0.00,0.00,0.000000,0.000000,0.000000,2.984000e+10,0.000000


In [7]:
creon = CREON()

In [8]:
for yyyy in "2017 2018 2019".split(" "):
    
    for mm in "1 2 3 4 5 6 7 8 9 10 11 12".split(" "):
        
        mm = mm.zfill(2)
        
        print(yyyy, mm)

        for idx, row in tqdm(KoreaStock.iterrows()):

            code = row["종목코드"]

            savedir = "/Users/juhy9/Documents/GitHub/Quant/data/{}".format(code)
            savefile = "{}/MIN_{}_{}_{}.txt".format(savedir, yyyy, mm, code)

            if not os.path.isdir(savedir):
                os.makedirs(savedir)

            if os.path.isfile(savefile):
                
                continue
                # os.remove(savefile)
                #read_table = pd.read_table(savefile, delimiter=" ")
                # read_table = pd.DataFrame([])
                
            else:
                read_table = pd.DataFrame([])

            creon.setMethod(code=code,
                            char="m",
                            from_yyyymmdd=int("{}{}31".format(yyyy, mm)),
                            to_yyyymmdd=int("{}{}01".format(yyyy, mm)))

            getStockPrice = creon.getStockPriceMin()
            
            DataFrame = pd.DataFrame(getStockPrice)
            DataFrame = DataFrame.iloc[::-1].reset_index(drop=True)

            if len(read_table) > 0:
                read_table = read_table.loc[~read_table.날짜.isin(DataFrame.날짜)].reset_index(drop=True)

            concat = pd.concat([read_table, DataFrame], 0).reset_index(drop=True)
            concat.to_csv(savefile, sep=" ", index=None)


2019 01


1551it [00:01, 1244.50it/s]


2019 02


1551it [00:00, 2098.31it/s]


2019 03


...

1486it [25:06, 13.07it/s]

15초당 60건으로 제한합니다.


1519it [25:18,  6.52it/s]

15초당 60건으로 제한합니다.


1551it [25:32,  1.01it/s]


In [9]:
for yyyy in "2020".split(" "):
    
    for mm in "1 2 3 4".split(" "):
        
        mm = mm.zfill(2)
        
        print(yyyy, mm)

        for idx, row in tqdm(KoreaStock.iterrows()):

            code = row["종목코드"]

            savedir = "/Users/juhy9/Documents/GitHub/Quant/data/{}".format(code)
            savefile = "{}/MIN_{}_{}_{}.txt".format(savedir, yyyy, mm, code)

            if not os.path.isdir(savedir):
                os.makedirs(savedir)

            if os.path.isfile(savefile):
                continue
                # os.remove(savefile)
                # read_table = pd.read_table(savefile, delimiter=" ")
                # read_table = pd.DataFrame([])
            else:
                read_table = pd.DataFrame([])

            creon.setMethod(code=code,
                            char="m",
                            from_yyyymmdd=int("{}{}31".format(yyyy, mm)),
                            to_yyyymmdd=int("{}{}01".format(yyyy, mm)))

            getStockPrice = creon.getStockPriceMin()
            
            DataFrame = pd.DataFrame(getStockPrice)
            DataFrame = DataFrame.iloc[::-1].reset_index(drop=True)

            if len(read_table) > 0:
                read_table = read_table.loc[~read_table.날짜.isin(DataFrame.날짜)].reset_index(drop=True)

            concat = pd.concat([read_table, DataFrame], 0).reset_index(drop=True)
            concat.to_csv(savefile, sep=" ", index=None)


2020 01


2it [00:02,  1.08s/it]

15초당 60건으로 제한합니다.


1365it [26:06,  7.46it/s]

15초당 60건으로 제한합니다.


...

1492it [28:25,  2.15it/s]

15초당 60건으로 제한합니다.


1524it [28:38,  4.94it/s]

15초당 60건으로 제한합니다.


1551it [28:52,  1.12s/it]


In [10]:
for yyyy in "2020".split(" "):
    
    for mm in "5 6 7".split(" "):
        
        mm = mm.zfill(2)
        
        print(yyyy, mm)

        for idx, row in tqdm(KoreaStock.iterrows()):

            code = row["종목코드"]

            savedir = "/Users/juhy9/Documents/GitHub/Quant/data/{}".format(code)
            savefile = "{}/MIN_{}_{}_{}.txt".format(savedir, yyyy, mm, code)

            if not os.path.isdir(savedir):
                os.makedirs(savedir)

            if os.path.isfile(savefile):
                continue
                # os.remove(savefile)
                # read_table = pd.read_table(savefile, delimiter=" ")
                # read_table = pd.DataFrame([])
            else:
                read_table = pd.DataFrame([])

            creon.setMethod(code=code,
                            char="m",
                            from_yyyymmdd=int("{}{}31".format(yyyy, mm)),
                            to_yyyymmdd=int("{}{}01".format(yyyy, mm)))

            getStockPrice = creon.getStockPriceMin()
            
            DataFrame = pd.DataFrame(getStockPrice)
            DataFrame = DataFrame.iloc[::-1].reset_index(drop=True)

            if len(read_table) > 0:
                read_table = read_table.loc[~read_table.날짜.isin(DataFrame.날짜)].reset_index(drop=True)

            concat = pd.concat([read_table, DataFrame], 0).reset_index(drop=True)
            concat.to_csv(savefile, sep=" ", index=None)


2020 05


3it [00:05,  1.87s/it]

15초당 60건으로 제한합니다.


1202it [25:40,  6.66it/s]

15초당 60건으로 제한합니다.


...

1502it [09:03,  8.48it/s]

15초당 60건으로 제한합니다.


1534it [09:18,  6.89it/s]

15초당 60건으로 제한합니다.


1551it [09:32,  2.71it/s]
